In [ ]:
import numpy as np
import felupe as fem
import matplotlib.pyplot as plt

import pyvista as pv
pv.set_jupyter_backend('trame')

In [ ]:
phi = np.linspace(1, 0.5, 21) * np.pi / 2
line = fem.mesh.Line(n=21)

In [ ]:
h = 1
L = 2
r = 0.3

In [ ]:
line.plot().show()

In [ ]:
curve = line.copy(points=r * np.vstack([np.cos(phi), np.sin(phi)]).T)
curve.plot().show()

In [ ]:
#np.vstack([np.linspace(0,h,21),np.linspace(h,h,21)]).T
top = line.copy(points=np.vstack([np.linspace(0,h,21),np.linspace(h,h,21)]).T)

In [ ]:
#test = np.linspace(0,1,21)**1.3*2-1
#test
face = curve.fill_between(top, n=np.linspace(0,1,21)**1.3*2-1)
face.plot().show()

In [ ]:
rect = fem.mesh.Rectangle(a=(h, 0), b=(L, h), n=21)
rect.plot().show()

In [ ]:
mesh = fem.mesh.concatenate([face, face.mirror(normal=[-1, 1, 0]), rect]) # concatenate these three pieces
mesh
#mesh.plot().show()

In [ ]:
mesh = mesh.sweep(decimals=5)
mesh
#mesh.plot().show()

In [ ]:
region = fem.RegionQuad(mesh)
displacement = fem.Field(region, dim=2)
field = fem.FieldContainer([displacement])

boundaries = fem.dof.symmetry(displacement)
boundaries.plot().show()

In [ ]:
umat = fem.LinearElasticPlaneStress(E=210000, nu=0.3)
solid = fem.SolidBody(umat, field)

In [ ]:
sig_inf=-100
region_boundary = fem.RegionQuadBoundary(mesh, mask=mesh.points[:, 0] == L)
field_boundary = fem.FieldContainer([fem.Field(region_boundary, dim=2)])

load = fem.SolidBodyPressure(field_boundary, pressure=sig_inf)

In [ ]:
step = fem.Step(items=[solid, load], boundaries=boundaries)
job = fem.Job(steps=[step]).evaluate()

solid.plot("Equivalent of Stress", show_edges=False, project=fem.topoints).show()

In [ ]:
plt.plot(
    fem.tools.project(solid.evaluate.stress(), region)[:, 0, 0][mesh.x == 0],
    mesh.points[:, 1][mesh.x == 0] / h,
    "o-",
)

plt.xlabel(r"$\sigma_{11}(x=0, y)$ in MPa $\longrightarrow$")
plt.ylabel(r"$y/h$ $\longrightarrow$")

In [ ]:
mesh

In [ ]:
mesh.save("partial_plate_with_hole.vtk")